## Transfer Learning Tuning:
  * model parameter and other information can be found below:
     * [source](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
     
  * base_model: use vgg16 and freeze at bottleneck layer (stop right before flatten layer) 
  * top_model: tune dense layers (parameters are inspired by source)
  
---
#### This cell is required in order to use GPU for running the model 

In [3]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import keras
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras
keras.backend.get_session().run(tf.global_variables_initializer())  

---
#### Import necessary libraries

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16
from keras import optimizers

---
#### Import train_df and test_df

In [6]:
train_df = pd.read_pickle('../pickle_files/train_df_glasses.pkl')
test_df = pd.read_pickle('../pickle_files/test_df_glasses.pkl')

In [8]:
train_df.shape[0]

505

#### Get bottleneck features to tune top models

In [9]:
def save_bottleneck_features(train_df, test_df, label, batch_size):
    '''
    inputs:
    train_df, test_df: train and test dataframes saved in pickle_files folder
    label: a string, eyewear, hat, or beard
    batch_size: process images in batches
    outputs:
    saves bottleneck features inside folder tuning_data as npy file
    '''
    # intialize the vgg16 model 
    # make sure not to train the top layers 
    base_model = VGG16(weights = 'imagenet', include_top = False)
    # create train_generator and test_generator to get bottleneck inputs for train and test df 
    # no image preprocessing other than rescale 
    datagen = ImageDataGenerator(rescale=1. / 255)
    # make sure shuffle is False so we know the label follows the sequence of the dataframe 
    # so we can tune top_model 
    train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../data/pics',
    x_col='pic_id',
    y_col=label,
    batch_size=batch_size,
    shuffle = False,
    target_size=(150,150),
    class_mode = None)
    # get features saved as .npy in tunign_data folder 
    bottleneck_features_train = base_model.predict_generator(
        train_generator, train_df.shape[0]//batch_size)
    np.save(open('../tuning_data/bottleneck_features_train.npy','wb'),
           bottleneck_features_train)
    
    test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='../data/pics',
    x_col='pic_id',
    y_col=label,
    batch_size=batch_size,
    shuffle = False,
    target_size=(150,150),
    class_mode = None)
    bottleneck_features_test = base_model.predict_generator(
        test_generator, test_df.shape[0]//batch_size)
    np.save(open('../tuning_data/bottleneck_features_test.npy','wb'),
           bottleneck_features_test)

#### Save bottleneck_features

In [13]:
save_bottleneck_features(train_df,test_df,'eyewear',96)

Found 505 validated image filenames.
Found 127 validated image filenames.


In [21]:
train_data = np.load(open('../tuning_data/bottleneck_features_train.npy','rb'))
train_label = np.array(train_df.eyewear.map({'not_'+'eyewear':0, 'eyewear':1}))
len(train_label)%96

25

### Quick tuning of top models 

In [31]:
def train_top_model(train_df, test_df, epoch, batch_size, label):
    '''
    inputs:
    train_df, test_df: dataframes saved in pickle_files to generate train and test labels 
    epoch: num of epochs in fit 
    batch_size: same as image generator batch size 
    label: a string, eyewear, hat, or beard
    output:
    saves model weights in a folder 
    '''
    train_data = np.load(open('../tuning_data/bottleneck_features_train.npy','rb'))
    # make sure train_data and train_label have same num of samples
    train_label = np.array(train_df.eyewear.map({'not_'+label:0, label:1}))[:-(train_df.shape[0]%batch_size)]
    
    test_data = np.load(open('../tuning_data/bottleneck_features_test.npy','rb'))
    test_label = np.array(test_df.eyewear.map({'not_'+label:0, label:1}))[:-(test_df.shape[0]%batch_size)]
    
    # build top model
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    
    model.fit(train_data, train_label,
             epochs=epoch,
             batch_size=batch_size,
             validation_data=(test_data,test_label))
    model.save_weights('../tuning_data/bottleneck_vgg_model.h5')

In [30]:
train_data = np.load(open('../tuning_data/bottleneck_features_train.npy','rb'))
label = 'eyewear'
train_label = np.array(train_df.eyewear.map({'not_'+label:0, label:1}))[:-(train_df.shape[0]%96)]
train_label

array([0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,

In [32]:
train_top_model(train_df, test_df, 50, 96, 'eyewear')

Train on 480 samples, validate on 96 samples
Epoch 1/50
480/480 [==============================] - 0s 687us/step - loss: 3.3371 - acc: 0.5625 - val_loss: 2.2513 - val_acc: 0.5000
Epoch 2/50
480/480 [==============================] - 0s 37us/step - loss: 1.8445 - acc: 0.5750 - val_loss: 0.4074 - val_acc: 0.8750
Epoch 3/50
480/480 [==============================] - 0s 39us/step - loss: 0.5149 - acc: 0.7771 - val_loss: 0.5397 - val_acc: 0.6771
Epoch 4/50
480/480 [==============================] - 0s 42us/step - loss: 0.6530 - acc: 0.7458 - val_loss: 0.3292 - val_acc: 0.9167
Epoch 5/50
480/480 [==============================] - 0s 41us/step - loss: 0.3033 - acc: 0.8708 - val_loss: 0.4934 - val_acc: 0.7604
Epoch 6/50
480/480 [==============================] - 0s 42us/step - loss: 0.5840 - acc: 0.7333 - val_loss: 0.2958 - val_acc: 0.9167
Epoch 7/50
480/480 [==============================] - 0s 44us/step - loss: 0.1975 - acc: 0.9375 - val_loss: 0.3794 - val_acc: 0.8125
Epoch 8/50
480/480 [===

In [3]:
# load train and test df for labels 
train_df = pd.read_pickle('finished_pickles/train_df_glasses.pkl')
test_df = pd.read_pickle('finished_pickles/test_df_glasses.pkl')

In [35]:
train_data = np.load(open('bottleneck_model_weights/bottleneck_features_train.npy','rb'))
train_label = np.array(train_df.eyewear.map({'Not_Glasses':0, 'Glasses':1}))
    
test_data = np.load(open('bottleneck_model_weights/bottleneck_features_test.npy','rb'))
test_label = np.array(test_df.eyewear.map({'Not_Glasses':0, 'Glasses':1}))[:-3]  # didn't run all 131 samples 

In [50]:
train_label[:-10].shape

(512,)

In [53]:
def train_top_model():
    train_data = np.load(open('bottleneck_model_weights/bottleneck_features_train.npy','rb'))
    train_label = np.array(train_df.eyewear.map({'Not_Glasses':0, 'Glasses':1}))[:-10]
    
    test_data = np.load(open('bottleneck_model_weights/bottleneck_features_test.npy','rb'))
    test_label = np.array(test_df.eyewear.map({'Not_Glasses':0, 'Glasses':1}))[:-3]
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    
    model.fit(train_data, train_label,
             epochs=50,
             batch_size=16,
             validation_data=(test_data,test_label))
    model.save_weights('bottleneck_model_weights/bottleneck_vgg_model.h5')

In [54]:
train_top_model()

Train on 512 samples, validate on 128 samples
Epoch 1/50
512/512 [==============================] - 0s 715us/step - loss: 1.4844 - acc: 0.6133 - val_loss: 0.5525 - val_acc: 0.6875
Epoch 2/50
512/512 [==============================] - 0s 199us/step - loss: 0.5770 - acc: 0.7188 - val_loss: 0.7599 - val_acc: 0.6406
Epoch 3/50
512/512 [==============================] - 0s 202us/step - loss: 0.4187 - acc: 0.8223 - val_loss: 0.8598 - val_acc: 0.6406
Epoch 4/50
512/512 [==============================] - 0s 195us/step - loss: 0.4532 - acc: 0.8008 - val_loss: 0.4884 - val_acc: 0.7812
Epoch 5/50
512/512 [==============================] - 0s 194us/step - loss: 0.2966 - acc: 0.8848 - val_loss: 0.5101 - val_acc: 0.7969
Epoch 6/50
512/512 [==============================] - 0s 190us/step - loss: 0.2718 - acc: 0.8945 - val_loss: 1.0055 - val_acc: 0.6250
Epoch 7/50
512/512 [==============================] - 0s 191us/step - loss: 0.2571 - acc: 0.8965 - val_loss: 0.4931 - val_acc: 0.7734
Epoch 8/50
512/5